# Import turbodbc Python module and pandas

In [1]:
#conda install -c conda-forge trubodbc
from turbodbc import connect, make_options
import pandas as pd

## Reading excel from path

In [2]:
df = pd.read_excel("https://s3-eu-west-1.amazonaws.com/londondatastore-upload/alternative-medal-ranks.xls")

## Define turbodbc module recomended settings

In [3]:
options = make_options(parameter_sets_to_buffer=1000, prefer_unicode=True)

## Define drive name, Server DB url, database name and account

In [4]:
conn = connect(driver='{ODBC Driver 13 for SQL Server}', server='den1.mssql5.gear.host',
                       database='cookbookdbname', 
                       TRUSTED_CONNECTION='no', turbodbc_options=options,uid="cookbookdbname", pwd="Hp7ah9Ezn!_K")

In [5]:
cursor = conn.cursor()

## Creating Table

In [6]:
cols = list(df.columns)
cols

['Basic Rank',
 'ISO',
 'Country',
 'Gold',
 'Silver',
 'Bronze',
 'Total',
 'Weighted total (Gold=3, Silver=2, Bronze=1)',
 '2011 population (from World Bank)',
 'Unnamed: 9',
 'Total medals per 10mn population',
 'Total weighted medals per 10mn pop',
 'Unnamed: 12',
 'Rank: Total medals per 10mn population',
 'Rank: Total weighted medals per 10mn pop']

In [8]:
query = """CREATE TABLE olimpiadas_prendas(
    Basic_Rank varchar(255),
    ISO varchar(255),
    Country varchar(255),
    Gold varchar(255),
    Silver varchar(255),
    Bronze varchar(255),
    Total varchar(255),
    W_T varchar(255),
    P_2011_World_Bank varchar(255),
    Unname varchar(255),
    T_M_10_P varchar(255),
    T_W_M_10_P varchar(255),
    unname2 varchar(255),
    R_T_M_10_p varchar(255),
    R_T_W_M_10_P varchar(255)
);"""

cursor.execute(query)
#cursor.execute("""DROP TABLE olimpiadas_prendas""")

## Execute many SQL querys

In [9]:
mis_preguntas = ",".join(['?' for i in cols])
mis_preguntas

'?,?,?,?,?,?,?,?,?,?,?,?,?,?,?'

In [10]:
insert_query = "insert into olimpiadas_prendas values({});".format(mis_preguntas)
insert_query

'insert into olimpiadas_prendas values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);'

In [11]:
data = df.values.tolist()
data = list(map(lambda x:list(map(str,x)),data))#each val to str

In [12]:
cursor.executemany(insert_query,data)

## Getting results

In [13]:
cursor.execute("select * from olimpiadas_prendas with(nolock)")

In [14]:
cursor.fetchall()

[['1',
  'USA',
  'United States',
  '46',
  '29',
  '29',
  '104',
  '225',
  '311591917.0',
  'nan',
  '3.3376989044295393',
  '7.220983206698523',
  'nan',
  '50',
  '46'],
 ['2',
  'CHN',
  'China',
  '38',
  '27',
  '22',
  '87',
  '190',
  '1344130000.0',
  'nan',
  '0.6472588216913542',
  '1.4135537485213483',
  'nan',
  '74',
  '71'],
 ['3',
  'GBR',
  'Great Britain',
  '29',
  '17',
  '19',
  '65',
  '140',
  '60924000.0',
  'nan',
  '10.669030267218174',
  '22.97944980631607',
  'nan',
  '22',
  '17'],
 ['4',
  'RUS',
  'Russian Federation',
  '24',
  '25',
  '33',
  '82',
  '155',
  '141930000.0',
  'nan',
  '5.777495948707109',
  '10.92087648840978',
  'nan',
  '33',
  '32'],
 ['5',
  'KOR',
  'Korea, Rep.',
  '13',
  '8',
  '7',
  '28',
  '62',
  '49779000.0',
  'nan',
  '5.624861889551819',
  '12.455051326864742',
  'nan',
  '34',
  '31'],
 ['6',
  'DEU',
  'Germany',
  '11',
  '19',
  '14',
  '44',
  '85',
  '81726000.0',
  'nan',
  '5.3838435748721345',
  '10.400606906

In [15]:
conn.commit()
conn.close()

In [16]:
# Turbodbc documentation http://turbodbc.readthedocs.io/en/latest/pages/introduction.html